# Protein preparation and solvation pyCHARMM example.
## Retreive protein molecule from RCSB (PDB)
## Build protein molecule (may be multiple chains) minimize, visualize
## Solvate in a box of water using MMTSB Toolset
## Proteins already available for this tutorial are PTI (pancreatic trypsin inhibitor), PDBIDs: 4pti and 6pti and HDEA, a bacterial pH stress protein, that is a dimer (two identical chains), PDBID: 5wyo.

### Note we use some specific integration of processing to retreive information about segnames used by MMTSB in spltting files and preparing pdb, searching for SSBONDs, determining the number of chains. While these work in the test cases here, they may not work for all proteins in the pdb.

### Note that the environment variable CHARMM_LIB_DIR must be defined. (Note: CHARMM_LIB_DIR should point to `<charmm_install_path>/lib`)

# pyCHARMM header files plus some of the necessary functionality

In [ ]:
# This script provides a simple example of building a
# protein and minimizing the structure and then
# calculating the energy to illustrate functionality to be
# exposed in pyCHARMM.
#  copyright C.L. Brooks III, June 1, 2022

# These are general python modules needed for this  tutorial
import os
import sys
import numpy as np

# These are a subset of the pycharmm modules that were installed when
# pycharmm was installed in your python environment
import pycharmm
import pycharmm.generate as gen
import pycharmm.ic as ic
import pycharmm.coor as coor
import pycharmm.energy as energy
import pycharmm.dynamics as dyn
import pycharmm.nbonds as nbonds
import pycharmm.minimize as minimize
import pycharmm.crystal as crystal
import pycharmm.image as image
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.cons_harm as cons_harm
import pycharmm.cons_fix as cons_fix
import pycharmm.select as select
import pycharmm.shake as shake

from pycharmm.lib import charmm as libcharmm


## Since this is a protein, let's run mmtsb.pl -info to check how many chains, then gather info on SSBONDs and use convpdb.pl to get separate pdb files for each chain and finally display the structure in pymol, including cystein residues.

In [ ]:
def get_assigned_segid(file=None):
    # read the pdb to get the assigned segid for each chain
    fpdb = open(file,'r')
    for l in fpdb:
        if l.split()[0] == 'ATOM':
            segid=l.strip().split()[-1]
            break
    fpdb.close()
    return segid

pdbid = '4pti'  # bovine pancreatic trypsin inhibitor - an x-ray structuture
#pdbid = '6pti'  # bovine pancreatic trypsin inhibitor - an x-ray structuture
#pdbid = '5wyo'  # HDEA a dimeric (two chain) protein - an NMR structure
# Check to see whether pdb directory needed below exists, if not, create it.
if not os.path.isdir('pdb'): os.system('mkdir pdb')
# Fetch the file from RCSB
!wget https://files.rcsb.org/download/'{pdbid}'.pdb -O pdb/'{pdbid}'.pdb
os.system(f'convpdb.pl -info pdb/{pdbid}.pdb > pdb/info')
os.system(f'grep SSBOND pdb/{pdbid}.pdb >> pdb/info')
fr = open('pdb/info', 'r')
chains = []
segids = []
disu = {}
for l in fr:
    if 'chains total' in l:
        nchains = l.split()[0]
    if l.split()[0]=='==':
        chains.append(l.split()[2])
    if l.split()[0]=='SSBOND':
        num = l.split()[1]
        res1 = l.split()[2]
        ch1 = l.split()[3]
        rn1 = l.split()[4]
        res2 = l.split()[5]
        ch2 = l.split()[6]
        rn2 = l.split()[7]
        disu[num] ={'residues':[res1,res2],
                    'chains':[ch1,ch2],
                    'resnums':[rn1,rn2]}
fr.close()
# Let's look at what was produced in the info file that we parsed
!cat pdb/info
os.system('rm pdb/info') # clean-up
for chain in chains:
    # 5wyo is an NMR structure w/ multiple models, we can probably deduce that from the pdb automatically, but here
    # we just do it by hand, pick the first model.
    if pdbid == '5wyo': 
        os.system(f'convpdb.pl -out charmm22 -nsel {chain}: -model 1 -segnames pdb/{pdbid}.pdb > pdb/{pdbid}_{chain.lower()}.pdb') # Note .lower() to fix filenames
    else: 
        os.system(f'convpdb.pl -out charmm22 -nsel {"protein"} -segnames pdb/{pdbid}.pdb > pdb/{pdbid}_{chain.lower()}.pdb')  # Note .lower() to fix filenames
    segids.append(get_assigned_segid(file=f'pdb/{pdbid}_{chain.lower()}.pdb'))

##  Read in the topology and parameter files
### The topology file contains the information pertinent to building molecular systems, either as independent molecules or as "residues" linked together to form more complex structures, i.e., proteins and nucleic acids. The parameter file contains the parameters that provide the information for the force field based calculations. CHARMM has topology and parameter files that are non-overlapping representations of different "regions" of chemical space, e.g., proteins ("_prot"), nucleic acids ("_na"), ethers, lipids, small drug-like molecules ("_cgenff"), etc. Here we will utilize the protein, water and ion topology and parameter files. These files are required to "generate" (see below) a psf that is necessary preceeding any molecular mechanics calculation.

In [ ]:
# Read in the topology (rtf) and parameter file (prm) for proteins
# equivalent to the CHARMM scripting command: read rtf card name toppar/top_all36_prot.rtf
read.rtf('../toppar/top_all36_prot.rtf')
# equivalent to the CHARMM scripting command: read param card flexible name toppar/par_all36m_prot.prm
read.prm('../toppar/par_all36m_prot.prm', flex=True)

# stream in the water/ions parameter using the pycharmm.lingo module
# equivalent to the CHARMM scripting command: stream toppar/toppar_water_ions.str
read.stream('../toppar/toppar_water_ions.str')
#pycharmm.lingo.charmm_script('stream toppar/toppar_water_ions.str')
# end toppar/toppar_water_ions.str


## Specify the sequence of residues, generate them and build them.
### To do this we need to input the sequence of "residues" we wish to do calculations on, generate the particlar sequence with any blocking groups needed, here we will terminate the sequence with an acetyl N-terminal "patch" and an N-methyl amide C-terminal "patch". Note we will use the segids just collected to construct the "segment" of structure for generation of the psf data structures in CHARMM we will setup the ic (internal coordinate) tables that will allow us to build the missing atoms in the structure from the known bond distances, angles and dihedrals.

In [ ]:
# Loop over the segments identified in the pdb files
for ichain,segid in enumerate(segids):
    # read in the sequence of the protein to be generated
    # only useful for the same residue
    # equivalent to the CHARMM scripting command:
    # read sequ pdb name pdb/{}_{}.pdb
    read.sequence_pdb(f'pdb/{pdbid}_{chains[ichain].lower()}.pdb') # Note .lower() to fix filenames

    # equivalent to the CHARMM scripting command: generate ADP first ACE last CT3 setup
    gen.new_segment(seg_name=segid, first_patch='ACE', last_patch='CT3', setup_ic=True)
    # equivalent to read coor pdb name pdb/{}_{}.pdb resid
    read.pdb(f'pdb/{pdbid}_{chains[ichain].lower()}.pdb',resid=True) # Note .lower() to fix filenames
    # equivalent to the CHARMM scripting command: ic param
    ic.prm_fill(replace_all=False)
    # equivalent to the CHARMM scripting command: ic build
    ic.build()
# add disulfide bonds in present
# note, we do this after we generate the sequences since there may be inter-chain disulfides
if len(disu.keys())>0:
    for ssbond in disu.keys():
        #pycharmm.lingo.charmm_script('patch disu pro{} {} pro{} {}'\
        #                             .format(disu[ssbond]['chains'][0],disu[ssbond]['resnums'][0],
        #                                     disu[ssbond]['chains'][1],disu[ssbond]['resnums'][1]))
        gen.patch('disu',
                  f'pro{disu[ssbond]["chains"][0]} {disu[ssbond]["resnums"][0]} pro{disu[ssbond]["chains"][1]} {disu[ssbond]["resnums"][1]}',
                  warn=True)
# The coor orie command is useful to expose since it allows one to
# orient the system in preparation for other calculations
# equivalent to the CHARMM scripting command: coor orient
coor.orient(by_rms=False,by_mass=False,by_noro=False)
# equivalent to the CHARMM scripting command: print coor
coor.show()
# If pdb directory doesn't already exist make it here.
if not os.path.isdir('pdb'): os.system('mkdir pdb')
# equivalent to the CHARMM scripting command: write coor pdb name pdb/pdbid_initial.pdb
write.coor_pdb(f'pdb/{pdbid}_initial.pdb')

## Visualize our initial structure to make sure it looks reasonable.

In [ ]:
import nglview as nv
view = nv.NGLWidget()
view.add_component(f'pdb/{pdbid}_initial.pdb')
view.clear_representations()
view.add_representation('cartoon',selection='not (water or ion)',color_scheme='resname')
atom_pair = []
if len(disu.keys())>0:
    view.add_representation('licorice',selection='CYS')
view.center()
view

## Now lets minimize the structure and view the minimized structure 
### To do this we will first specify the non-bonded parameters we want to use for these molecular mechanics calculations. Note we are spcifying that we will use the fswitch and vfswitch force-switching functions to truncate the electrostatic and vdW interactions. The switching will occur between ctonnb and ctofnb. We will build a non-bonded list to process pairs of interacting atoms using a cutoff radius of cutnb. We will use atom-based electrostatic (atom) and vdW (vatom) non-bonded list generaton schemes and will repreent the Coulomb electrostatic interactions with a constant dielectric constant (versus an r-dependent dielectric constant) of 1 (cdie=1.0). There are multiple ways of inputting the nonbonded parameters, here we create a python object my_nbonds which has a .run() attribute. 
### We set harmonic restraints on the CA atoms and allow the backbone to relax.
### Next we will use CHARMM's sd (steepest descents) minimizer to minimize the structure we just built using the non-bonded methods we just specified. The minimizer is set-up to run 300 steps of minimization unless the energy change falls below 0.001 or the gradient of the energy falls below 0.001, in which case minimization will stop. The purpose is to just get any "vdW bumps" cleaned-up and bond lengths/angles adjusted to our potential function. Finally we print out the final energy.

In [ ]:
# Specify nonbonded python object called my_nbonds - this just sets it up
# equivalant CHARMM scripting command: nbonds cutnb 18 ctonnb 13 ctofnb 17 cdie eps 1 atom vatom fswitch vfswitch
my_nbonds = pycharmm.NonBondedScript(
    cutnb=18.0, ctonnb=13.0, ctofnb=17.0,
    eps=1.0,
    cdie=True,
    atom=True, vatom=True,
    fswitch=True, vfswitch=True)

# Implement these non-bonded parameters by "running" them.
my_nbonds.run()
# equivalent to: cons harm force 20 select type ca end
cons_harm.setup_absolute(selection=pycharmm.SelectAtoms(atom_type='CA'),force_constant=20)
# equivalent CHARMM scripting command: minimize abnr nstep 1000 tole 1e-3 tolgr 1e-3
minimize.run_sd(nstep=300, tolenr=1e-3, tolgrd=1e-3)
# equivalent CHARMM scripting command: energy
energy.show()
# equivalent to the CHARMM scripting command: write coor pdb name pdb/initial.pdb
write.coor_pdb(f'pdb/{pdbid}_minimized.pdb'.format(pdbid))
write.psf_card(f'pdb/{pdbid}.psf')

## Visually compare the minimized and initial structure? Do they look different?

In [ ]:
view_1 = nv.NGLWidget()
view_1.add_component(f'pdb/{pdbid}_initial.pdb')
view_1.clear_representations()
view_1.add_representation('cartoon',selection='not (water or ion)',color_scheme='resname')
view_1.add_representation('licorice',selection='not (water or ion)',color='lightgreen')
view_1.add_component(f'pdb/{pdbid}_minimized.pdb')
view_1.add_representation('licorice',selection='not (water or ion)',color='lightblue',component=1)
if len(disu.keys())>0:
    view_1.add_representation('licorice',selection='CYS',component=1,color='lightblue')
view_1.center()
view_1

# Solvate the protein in TIP3P water
## In the following we will use the MMTSB toolset to solvate the blocked alanine residue in a cubic box of TIP3P water using the convpdb.pl commands noted below.

## First we figure out whether we need to add counter ions to neutralize the overall charge.

In [ ]:
import numpy as np
# find the overall charge so we can add neutralizing ions
q = psf.get_charges()
Ntot = round((np.sum(q)))
if Ntot > 0: ion_type = 'CLA'
if Ntot < 0: ion_type = 'SOD'
ions = '-ions {}:{}'.format(ion_type,np.abs(Ntot))
if np.abs(Ntot) < 1e-2: ions = ''

In [ ]:
# CHARMM scripting command: system "convpdb.pl -solvate {ions} -cutoff 10 -cubic -out charmm22 pdb/adp.pdb
# | convpdb.pl -segnames -nsel TIP3 > pdb/{}_wt00.pdb"
solvate_command = 'convpdb.pl -solvate -cutoff 10 {} -cubic -out charmm22 pdb/{}_minimized.pdb > pdb/w.pdb;'\
    .format(ions,pdbid)
solvate_command +='convpdb.pl -segnames -nsel TIP3 pdb/w.pdb | '
solvate_command +=f'sed "s/WT0[1,2,3,4,5]/WT00/g" > pdb/{pdbid}_wt00.pdb;'
solvate_command +='convpdb.pl -segnames -nsel ion pdb/w.pdb > pdb/ions.pdb'
# run the command as a system subprocess
os.system(solvate_command)
# replace HETATM by ATOM in ions
fpdb = open('pdb/ions.pdb','r')
opdb = open(f'pdb/{pdbid}_ions.pdb','w')
for l in fpdb:
    print(l.strip().replace('HETATM','ATOM  '),file=opdb)
fpdb.close()
opdb.close()
# clean-up non-specific files
os.system('rm pdb/ions.pdb')

In [ ]:
solvated_1 = nv.NGLWidget()
solvated_1.add_component(f'pdb/{pdbid}_minimized.pdb')
solvated_1.clear_representations()
solvated_1.add_representation('cartoon',selection='protein', color_scheme='resname')
solvated_1.add_representation('surface',surfaceType='ms',opacity=0.5,selection='protein', color='lightblue')
if len(disu.keys())>0:
    solvated_1.add_representation('licorice',selection='CYS')
solvated_1.add_component(f'pdb/{pdbid}_ions.pdb')
solvated_1.clear_representations(component=1)
solvated_1.add_representation('spacefill',selection='CLA', color='yellow',component=1)
solvated_1.add_representation('spacefill',selection='SOD or POT', color='red',component=1)
solvated_1.add_component(f'pdb/{pdbid}_wt00.pdb')
solvated_1.clear_representations(component=2)
solvated_1.add_representation('licorice',selection='water',opacity=0.4, component=2)
solvated_1.center()
solvated_1

### Generate water segment and minimize the system, protein + solvent + ions, finally save the psf and coordinates
### Note that in "conditioning" the system I first fix the protein atoms and then minimize the water. I am using the steepest descents algorithm because it works best for large systems and/or when you may have bad contacts.

1) **build the water and ion segments**

In [ ]:
# Here is an alternative means of reading a sequence
# read sequ pdb name pdb/{}_wt00.pdb
# get the water segid:
water_segment = get_assigned_segid(file=f'pdb/{pdbid}_wt00.pdb')
# Let's set the wrnlev to 0 to avoid the large output
old_wrnlev = settings.set_warn_level(0)
read.sequence_pdb(f'pdb/{pdbid}_wt00.pdb')
# Now reset back to default wrnlev
settings.set_warn_level(old_wrnlev)
# Another example of the generate command
# generate wt00 noangle nodihedral
gen.new_segment(water_segment, angle=False, dihedral=False)

# read coor pdb name pdb/pdbid_wt00.pdb resid
read.pdb(f'pdb/{pdbid}_wt00.pdb', resid=True)

# Here is an alternative means of reading a sequence
# read sequ pdb name pdb/{}_ions.pdb
# get ion sequence name
ion_segment = get_assigned_segid(file=f'pdb/{pdbid}_ions.pdb'.format(pdbid))
read.sequence_pdb(f'pdb/{pdbid}_ions.pdb')

# Another example of the generate command
# generate wt00 noangle nodihedral
gen.new_segment(ion_segment, angle=False, dihedral=False)

# read coor pdb name pdb/adp.pdb resid
read.pdb(f'pdb/{pdbid}_ions.pdb', resid=True)
# get the coor statistics to construct boxlengths
# coor stat
stats = coor.stat()

2) **Figure out the boxsize**
3) **add periodic boundary conditions**
4) **minimize**
5) **write psf and final minimized coordinates, ready for simulations**

In [ ]:
# boxsize
xsize = stats['xmax'] - stats['xmin']
ysize = stats['ymax'] - stats['ymin']
zsize = stats['zmax'] - stats['zmin']
boxsize = max(xsize, ysize, zsize)

# half box size
boxhalf = boxsize / 2.0

# CHARMM scripting: crystal define cubic @boxsize @boxsize @boxsize 90 90 90
crystal.define_cubic(boxsize)
# CHARMM scripting: crystal build cutoff @boxhalf noper 0
crystal.build(boxhalf)

# Turn on image centering - bysegment for protein, by residue for solvent and ions
# CHARMM scripting: image byseg xcen 0 ycen 0 zcen 0 select segid SEGID end
for segid in segids:
    image.setup_segment(0.0, 0.0, 0.0, segid)
# CHARMM scripting: image byres xcen 0 ycen 0 zcen 0 select resname tip3 end
image.setup_residue(0.0, 0.0, 0.0, 'TIP3')
# CHARMM scripting: image byres xcen 0 ycen 0 zcen 0 select resname ion_type end
image.setup_residue(0.0, 0.0, 0.0, ion_type)

# Now specify nonbonded cutoffs for solvated box
cutnb = min(boxhalf,12)
cutim = cutnb
ctofnb = cutnb - 1.0
ctonnb = cutnb - 3.0

# Another nbonds example
# CHARMM scripting: nbonds cutnb @cutnb cutim @cutim ctofnb @ctofnb ctonnb @ctonnb -
#        inbfrq -1 imgfrq -1
pycharmm.NonBondedScript(
    cutnb=cutnb, cutim=cutim, ctonnb=ctonnb, ctofnb=ctofnb,
    eps=1.0,
    cdie=True,
    atom=True, vatom=True,
    fswitch=True, vfswitch=True,
    inbfrq=-1, imgfrq=-1).run()

# Fix the peptide and minimize the solvent to "fit"
# CHARMM scripting: cons fix select segid adp end
cons_fix.setup(pycharmm.SelectAtoms(seg_id=segids))

# Minimize the solvent positions with periodic boundary conditions using steepest descents
# CHARMM scripting: mini sd nstep 200 tole 1e-3 tolgrd 1e-3
minimize.run_sd(nstep=200, tolenr=1e-3, tolgrd=1e-3)

# Turn off fixed atoms
# CHARMM scripting: cons fix select none end
cons_fix.turn_off()

# Write the psf and coordinates for the solvated peptide
# write psf card name pdb/adp+wat.psf
write.psf_card(f'pdb/{pdbid}+wat.psf')
# write coor pdb name pdb/adp+wat_min.pdb
write.coor_pdb(f'pdb/{pdbid}+wat_min.pdb')

## Finally, let's visualize the system and see what things look like!

In [ ]:
solvated = nv.NGLWidget()
solvated.add_component(f'pdb/{pdbid}_minimized.pdb')
solvated.clear_representations()
solvated.add_representation('cartoon',selection='protein', color_scheme='resname')
solvated.add_representation('surface',surfaceType='ms',opacity=0.5,selection='protein', color='lightblue')
if len(disu.keys())>0:
    solvated.add_representation('licorice',selection='CYS')
solvated.add_component(f'pdb/{pdbid}_ions.pdb')
solvated.clear_representations(component=1)
solvated.add_representation('spacefill',selection='CLA', color='green',component=1)
solvated.add_representation('spacefill',selection='SOD or POT', color='red',component=1)
solvated.add_component(f'pdb/{pdbid}_wt00.pdb')
solvated.clear_representations(component=2)
solvated.add_representation('licorice',selection='water',component=2)
solvated.center()
solvated

# This is the end of this tutorial example. 
## You should have learned how to 1) build a multi-segment protein with disulfide bonds; 2) to minimize the system and examine the results of the minimization; 3) to use the MMTSB Toolset to solvate and neutralize the system (with convpdb.pl); 4) to prepare the solvated system for further calculations by "conditioning" the solvent after the solvent overlay.
## As an exercise, try modifying this tutorial to build your favorite protein. Use the same blocking groups.

### Other things to consider are:
1) How do we choose protonation state for fixed protonation state models?
* use knowledge of pKas of titratable residues in protein of interest
* use a tool like PropKa to assign appropriate protonation state for titratable residues
2) How do we add a specific ionic strength of excess salt?